In [1]:
import math


class tensor:
    def __init__(self, data, _children=(), _op=""):
        if (isinstance(data, list)):
            return [tensor(i) for i in data]
        
        self.data = data
        self._prev = set(_children)
        self._backward = lambda: None
        self._op = _op
        self.grad = 0.0

    def __repr__(self):
        return f"Tensor[data = {self.data}]"

    def __add__(self, other):
        if isinstance(other, tensor) == False:
            other = tensor(other)
        out = tensor(self.data + other.data, (self, other), "+")

        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad

        out._backward = _backward
        return out

    def __mul__(self, other):
        if isinstance(other, tensor) == False:
            other = tensor(other)
        out = tensor(self.data * other.data, (self, other), "*")

        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad

        out._backward = _backward
        return out

    def __pow__(self, other):
        assert isinstance(other, (int, float))
        out = tensor(self.data**other, (self,), f"**{other}")

        def _backward():
            self.grad += (other * self.data ** (other - 1)) * out.grad

        out._backward = _backward
        return out

    def __neg__(self):
        return self * -1

    def __sub__(self, other):
        return self + (-other)

    def __rmul__(self, other):
        return self * other

    def __truediv__(self, other):
        return self * other**-1

    def tanh(self):
        x = self.data
        out = (math.exp(2.0 * x) - 1) / (math.exp(2.0 * x) + 1)
        out = tensor(out, (self,), "tanh")

        def _backward():
            self.grad += (1 - out.data**2) * out.grad

        out._backward = _backward
        return out

    def exp(self):
        x = self.data
        out = tensor(math.exp(x), (self,), "exp")

        def _backward():
            self.grad += out.data * out.grad

        out._backward = _backward
        return out

    def backward(self):
        self.grad = 1.0

        nodes = []
        added_nodes = set()

        def get_child_nodes(v):
            if v not in added_nodes:
                added_nodes.add(v)
                for child in v._prev:
                    get_child_nodes(child)
                nodes.append(v)

        get_child_nodes(self)

        for node in reversed(nodes):
            node._backward()
            
    def shape():
        return None

In [2]:
a = tensor(2)
b = tensor(0.25)
c = tensor(1)
e = tensor(-0.5)

In [3]:
t = (a * b) * e
l = t + c
o = l.tanh()

In [49]:
import torch

In [86]:
t_a = torch.Tensor([2]).double()
t_b = torch.Tensor([0.25]).double()
t_c = torch.Tensor([1]).double()
t_e = torch.Tensor([-0.5]).double()

In [87]:
t_a.requires_grad = True
t_b.requires_grad = True
t_c.requires_grad = True
t_e.requires_grad = True

In [88]:
t_o = torch.tanh((t_a * t_b) * t_e + t_c)

In [89]:
t_o.backward()

In [90]:
print(t_o.grad)
print(t_c.grad)
print(t_a.grad)
print(t_b.grad)
print(t_e.grad)

None
tensor([0.5966], dtype=torch.float64)
tensor([-0.0746], dtype=torch.float64)
tensor([-0.5966], dtype=torch.float64)
tensor([0.2983], dtype=torch.float64)


C:\Users\Administrator\AppData\Local\Temp\ipykernel_89740\1868670993.py:1: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at aten\src\ATen/core/TensorBody.h:494.)
  print(t_o.grad)


In [93]:
o

Tensor[data = 0.6351489523872873]

In [94]:
o.backward()

In [95]:
print(o.grad)
print(l.grad)
print(c.grad)
print(a.grad)
print(b.grad)
print(e.grad)

1.0
0.5965858082813315
0.5965858082813315
-0.07457322603516643
-0.5965858082813315
0.2982929041406657


In [64]:
def tr():
    h = 0.0001

    L2 = ((a + h) * b) * e + c

    L1 = (a * b) * e + c

    print((L2.data - L1.data) / h)


tr()

-40.000000000048885


In [ ]:
import random

class nn:
    def Linear(in_features, out_features):
        w = [[random.random() for _ in in_features] for _ in out_features]
        print(w)